<a href="https://colab.research.google.com/github/Computer-CGuy/MusgraveTextureNeuralRenderer/blob/models/SKIN_CANCER_%7C_NeuralRenderer_0.23_L1Loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch  
import matplotlib.pyplot as plt  
import numpy as np  
import torch.nn.functional as func
import torch.nn.functional as F
import PIL.ImageOps  
from torch import nn  
from torchvision import datasets,transforms   
import requests  
from PIL import Image  
import torchvision.models as models
import os
import json
from torch.utils.data import Dataset, DataLoader, Subset
import cv2
from google.colab.patches import cv2_imshow
import torchvision
import pickle

In [2]:
net = models.resnext50_32x4d().cuda()

In [3]:
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [4]:
net.fc = nn.Sequential(
          nn.Linear(2048,1024),
          nn.LeakyReLU(),
          nn.Linear(1024,4),
          nn.LeakyReLU()
        ).cuda()

In [5]:
from PIL import Image

In [ ]:
!ls "/content/drive/My Drive/MusgraveTextureNeuralRenderer/" | grep *.pkl

In [12]:
with open("/content/drive/My Drive/MusgraveTextureNeuralRenderer/GeneratorDictionaries/out.pkl","rb") as f:
    l = (pickle.load(f))

In [11]:
!ls "/content/drive/My Drive/MusgraveTextureNeuralRenderer/GeneratorDictionaries/" | grep ".*pkl"

out.pkl


In [10]:
!ls "/content/drive/My Drive/*/" | wc -l

ls: cannot access '/content/drive/My Drive/*/': No such file or directory
0


In [13]:
class MusgraveDataset(Dataset):
    def __init__(self,transforms):
        with open("/content/drive/My Drive/MusgraveTextureNeuralRenderer/GeneratorDictionaries/out.pkl","rb") as f:
            self.l = (pickle.load(f))
        self.transforms = transforms
    def __getitem__(self, index):
        # print(index)
        l = self.l
        file_name = l[index]["name"]
        file_name = "/content/drive/My Drive/MusgraveTextureNeuralRenderer/GeneratorDictionaries/"+file_name
        img = Image.open(file_name)
        # img = cv2.imread(file_name)
        # img = np.transpose(img,(2,0,1))
        # transforms.Compose([
            
        # ])
        return ((self.transforms(img),torch.Tensor(l[index]["values"])))
    
    def __len__(self):
        return len(self.l)

In [ ]:
# !ls "./drive/My Drive/MusgraveTextureNeuralRenderer/Generator/*.pkl"

In [ ]:
# import pickle
# with open("./drive/My Drive/MusgraveTextureNeuralRenderer/out.pkl","rb") as f:
#     l = (pickle.load(f))

In [14]:
transform = transforms.Compose([torchvision.transforms.Resize((224,224)),transforms.ToTensor()])
train = MusgraveDataset(transforms = transform)
train_loader = DataLoader(dataset=train, batch_size=10, shuffle=True, num_workers=2)

In [15]:
import torch.optim as optim

# criterion = nn.MSELoss(reduction='sum')
criterion = nn.L1Loss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

# optimizer = optim.SGD(render.parameters(), lr=0.001, momentum=0.9)

In [16]:
def show(img):
    img = img.detach().cpu().numpy()[0]
    img  = np.transpose(img,(1,2,0))
    print(img.shape)
    cv2_imshow(img)

In [ ]:
# train.l

In [ ]:
# inputs.shape

In [18]:
loss_n = None
for epoch in range(3):
    for i,data in enumerate(train_loader):
        inputs, labels = data[0].cuda(),data[1].cuda()
        labels[:,0]/=12
        labels[:,1]/=16
        labels[:,2]/=4
        labels[:,3]/=3
        # labels/=255
        optimizer.zero_grad()
        
        output = net(inputs)
        
        loss = criterion(output, labels)
        try:
            loss_n+=loss
        except:
            loss_n = loss
        
        if(i%10==9):
            loss_n.backward()
            optimizer.step()
            # print(loss.item())
            print(loss_n.item()/10)
            # print(output,inputs)
            loss_n = None
        
        

0.4613002300262451
0.6032241821289063
0.4466105937957764
0.34648966789245605
0.4042983531951904
0.39750032424926757
0.38542680740356444
0.37487139701843264
0.3532769441604614
0.34022233486175535
0.31081764698028563
0.28945834636688234
0.2984044075012207
0.3199914455413818
0.28416550159454346
0.2728682518005371
0.28348889350891116
0.28758046627044676
0.28121631145477294
0.3039703845977783
0.27675135135650636
0.2748187780380249
0.27446234226226807
0.28197789192199707
0.2586889028549194
0.2536175727844238
0.24330682754516603
0.25835769176483153
0.2742879867553711
0.25856432914733884
0.2741560459136963
0.2583728313446045
0.2545945167541504
0.2495429277420044
0.23798742294311523
0.24157350063323973
0.24654045104980468
0.26871581077575685
0.24240541458129883
0.23125741481781006
0.24958369731903077
0.25601658821105955
0.2596899509429932
0.26251399517059326
0.2685851097106934
0.2642462491989136
0.2539353370666504
0.2528287410736084
0.2462860107421875
0.2481750726699829
0.2540693759918213
0.233

KeyboardInterrupt: ignored

In [19]:
torch.save(net, "/content/drive/My Drive/MusgraveTextureNeuralRenderer/model1.pth")

In [ ]:
output

tensor([[0.7174, 0.6594, 0.2193, 0.8613],
        [0.1891, 0.7137, 0.7250, 0.3613],
        [0.6512, 0.1463, 0.6032, 0.5859],
        [0.4929, 0.5860, 0.7239, 0.1767],
        [0.5161, 0.8120, 0.5533, 0.3804],
        [0.5530, 0.7010, 0.5799, 0.6499],
        [0.5630, 0.2716, 0.5716, 0.6750],
        [0.5287, 0.3156, 0.7957, 0.5115],
        [0.5720, 0.4689, 0.5207, 0.7618],
        [0.5846, 0.2729, 0.7457, 0.2174]], device='cuda:0',
       grad_fn=<AddmmBackward>)

In [ ]:
labels

tensor([[0.1667, 0.5625, 0.0000, 1.0000],
        [0.2500, 0.6875, 0.7500, 0.0000],
        [0.3333, 0.1875, 0.7500, 0.3333],
        [1.0000, 0.5625, 0.5000, 0.0000],
        [0.5833, 0.5000, 0.5000, 0.0000],
        [0.2500, 0.8125, 1.0000, 0.6667],
        [0.6667, 0.3750, 0.7500, 1.0000],
        [0.0833, 0.3125, 1.0000, 0.3333],
        [0.8333, 0.5625, 0.5000, 1.0000],
        [0.2500, 0.0625, 1.0000, 0.0000]], device='cuda:0')

In [ ]:
torch.save(render.state_dict(), "./drive/My Drive/Skin Cancer/RenderResNet.pth")

In [ ]:
torch.save(render.state_dict(), "./drive/My Drive/Skin Cancer/AlexNet.pth")

In [ ]:
def norm(out):
    out-=out.mean()
    out/=out.max()
    return out

In [ ]:
inputs.shape,output.shape

(torch.Size([4, 4]), torch.Size([4, 1000]))

In [ ]:
NetEpochs = 10000
start, stop, step = 7,10,1
Dataset = [x for x in range(1000)]
# Dataset = [0,1]
# Dataset = [0]
done = False
for data in Dataset:
  
  tdata = torch.Tensor([p]).cuda()
  X = model(tdata)
  # print(X)t
  X_CNN = cnn(X)
  # X = X*(255/X.mean())

  X_CNN2 = cnn2(X)
  
  img = process_image(X)
  # print(data,end=" ")
  # print("[ AlexNet "+str(dat[int(torch.argmax(X_CNN).detach().cpu().numpy())]),end="] ")
  # print("[ ResNet "+str(dat[int(torch.argmax(X_CNN2).detach().cpu().numpy())]),end="]")
  
  # cv2_imshow(img)
  cv2.imwrite("dog.jpg",img)
  os.system("python gradcam.py --image-path=dog.jpg --use-cuda")
  cp(data)
  updateJson(data,X_CNN,X_CNN2)
  if((data%10)==0):
    print(data)
  # break
  # imgR = cv2.imread("cam.jpg")
  # cv2_imshow(imgR)
